1. load all csv files into one data frame 
2. clean data by removing non-essential features (data known after the match, or irrelevant information)
3. clean data by removing data instances that have NaN for an essential feature
4. use feature engineering to create new features for each instance. make this into a new dataframe
5. train several different models and test the accuracy for each one
    a. random forest
    b. logistic regression
    c. SVM
    d. deep neural network
6. calculate expected value per match
7. maybe try bringing in more features like H2H, ELO, ELO on different surfaces, win % on surface, wins per last 50 games, etc.

In [15]:
import pandas as pd

path = '/Users/maxharsh/Desktop/Coding Projects/MatchNet/data/atp_matches_'
df = pd.DataFrame()

for year in range(2000, 2025):
    if year != 2020:
        file = path + str(year) + ".csv"
        year_data = pd.read_csv(file)
        df = pd.concat([df, year_data], axis=0)
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,NaN,...,55.0,39.0,29.0,17.0,4.0,7.0,11.0,1612.0,63.0,595.0
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,32.0,25.0,18.0,12.0,3.0,6.0,211.0,157.0,49.0,723.0
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,NaN,...,33.0,20.0,7.0,8.0,7.0,11.0,48.0,726.0,59.0,649.0
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,NaN,...,43.0,29.0,14.0,10.0,6.0,8.0,45.0,768.0,61.0,616.0
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,46.0,34.0,18.0,12.0,5.0,9.0,167.0,219.0,34.0,873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,2024-M-DC-2024-WG2-PO-URU-MDA-01,Davis Cup WG2 PO: URU vs MDA,Clay,4,D,20240203,5,212051,NaN,NaN,...,30.0,17.0,7.0,6.0,8.0,14.0,1109.0,8.0,740.0,34.0
3072,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,1,122533,NaN,NaN,...,41.0,25.0,6.0,9.0,1.0,4.0,554.0,67.0,748.0,32.0
3073,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,2,144748,NaN,NaN,...,51.0,25.0,7.0,11.0,5.0,12.0,416.0,109.0,NaN,NaN
3074,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,Hard,4,D,20240202,4,122533,NaN,NaN,...,51.0,32.0,17.0,14.0,5.0,9.0,554.0,67.0,416.0,109.0


In [16]:
# Remove all columns that are not useful
columns_to_keep = [
    'tourney_id', 'tourney_name', 'surface', 'tourney_level', 'tourney_date',
    'winner_name', 'loser_name', 'round', 'best_of',
    'winner_rank', 'loser_rank', 'winner_rank_points', 'loser_rank_points',
    'winner_hand', 'loser_hand', 'winner_ht', 'loser_ht', 'winner_age', 'loser_age'
]

df = df[columns_to_keep]
print("After removing extraneous columns:", df.shape)

# Drop rows with any missing values in those columns
df = df.dropna()

# Optionally, reset index after dropping
df = df.reset_index(drop=True)
print("After removing extraneous rows:", df.shape)
df.head(10)


After removing extraneous columns: (73444, 19)
After removing extraneous rows: (69744, 19)


,tourney_id,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,best_of,winner_rank,loser_rank,winner_rank_points,loser_rank_points,winner_hand,loser_hand,winner_ht,loser_ht,winner_age,loser_age
0,2000-301,Auckland,Hard,A,20000110,Tommy Haas,Jeff Tarango,R32,3,11.0,63.0,1612.0,595.0,R,L,188.0,180.0,21.7,31.1
1,2000-301,Auckland,Hard,A,20000110,Juan Balcells,Franco Squillari,R32,3,211.0,49.0,157.0,723.0,R,L,190.0,183.0,24.5,24.3
2,2000-301,Auckland,Hard,A,20000110,Alberto Martin,Alberto Berasategui,R32,3,48.0,59.0,726.0,649.0,R,R,175.0,173.0,21.3,26.5
3,2000-301,Auckland,Hard,A,20000110,Juan Carlos Ferrero,Roger Federer,R32,3,45.0,61.0,768.0,616.0,R,R,183.0,185.0,19.9,18.4
4,2000-301,Auckland,Hard,A,20000110,Michael Sell,Nicolas Escude,R32,3,167.0,34.0,219.0,873.0,R,R,180.0,185.0,27.3,23.7
5,2000-301,Auckland,Hard,A,20000110,Michael Chang,Byron Black,R32,3,50.0,70.0,722.0,563.0,R,R,175.0,175.0,27.8,30.2
6,2000-301,Auckland,Hard,A,20000110,Magnus Gustafsson,Mark Nielsen,R32,3,60.0,246.0,626.0,135.0,R,R,185.0,185.0,33.0,22.2
7,2000-301,Auckland,Hard,A,20000110,Thomas Johansson,Glenn Weiner,R32,3,43.0,334.0,785.0,88.0,R,R,180.0,188.0,24.7,23.7
8,2000-301,Auckland,Hard,A,20000110,Sjeng Schalken,Goran Ivanisevic,R32,3,37.0,68.0,843.0,571.0,R,L,193.0,193.0,23.3,28.3
9,2000-301,Auckland,Hard,A,20000110,Tomas Behrend,Markus Hantschk,R32,3,115.0,95.0,344.0,416.0,R,R,193.0,188.0,25.0,22.1


In [17]:
# Code to transform data
import numpy as np

# Same seed for reproducibility
np.random.seed(42) 

# Generate random 0 or 1: 0 → winner is Player 0, 1 → loser is Player 0
player0_is_winner = np.random.randint(0, 2, size=len(df))

# Build new DataFrame with properly assigned player fields
df_transformed = pd.DataFrame({
    # Player 0 fields (randomly chosen winner or loser)
    'player0_name': df['winner_name'].where(player0_is_winner == 0, df['loser_name']),
    'player0_rank': df['winner_rank'].where(player0_is_winner == 0, df['loser_rank']),
    'player0_rank_points': df['winner_rank_points'].where(player0_is_winner == 0, df['loser_rank_points']),
    'player0_hand': df['winner_hand'].where(player0_is_winner == 0, df['loser_hand']),
    'player0_ht': df['winner_ht'].where(player0_is_winner == 0, df['loser_ht']),
    'player0_age': df['winner_age'].where(player0_is_winner == 0, df['loser_age']),

    # Player 1 fields (opposite of above)
    'player1_name': df['loser_name'].where(player0_is_winner == 0, df['winner_name']),
    'player1_rank': df['loser_rank'].where(player0_is_winner == 0, df['winner_rank']),
    'player1_rank_points': df['loser_rank_points'].where(player0_is_winner == 0, df['winner_rank_points']),
    'player1_hand': df['loser_hand'].where(player0_is_winner == 0, df['winner_hand']),
    'player1_ht': df['loser_ht'].where(player0_is_winner == 0, df['winner_ht']),
    'player1_age': df['loser_age'].where(player0_is_winner == 0, df['winner_age']),

    # Match info
    'tourney_id': df['tourney_id'],
    'tourney_name': df['tourney_name'],
    'surface': df['surface'],
    'tourney_level': df['tourney_level'],
    'tourney_date': df['tourney_date'],
    'round': df['round'],
    'best_of': df['best_of'],
})

# Label: 1 if Player 0 is winner, else 0
df_transformed['label'] = (df_transformed['player0_name'] != df['winner_name']).astype(int)

df_transformed.head(10)

,player0_name,player0_rank,player0_rank_points,player0_hand,player0_ht,player0_age,player1_name,player1_rank,player1_rank_points,player1_hand,player1_ht,player1_age,tourney_id,tourney_name,surface,tourney_level,tourney_date,round,best_of,label
0,Tommy Haas,11.0,1612.0,R,188.0,21.7,Jeff Tarango,63.0,595.0,L,180.0,31.1,2000-301,Auckland,Hard,A,20000110,R32,3,0
1,Franco Squillari,49.0,723.0,L,183.0,24.3,Juan Balcells,211.0,157.0,R,190.0,24.5,2000-301,Auckland,Hard,A,20000110,R32,3,1
2,Alberto Martin,48.0,726.0,R,175.0,21.3,Alberto Berasategui,59.0,649.0,R,173.0,26.5,2000-301,Auckland,Hard,A,20000110,R32,3,0
3,Juan Carlos Ferrero,45.0,768.0,R,183.0,19.9,Roger Federer,61.0,616.0,R,185.0,18.4,2000-301,Auckland,Hard,A,20000110,R32,3,0
4,Michael Sell,167.0,219.0,R,180.0,27.3,Nicolas Escude,34.0,873.0,R,185.0,23.7,2000-301,Auckland,Hard,A,20000110,R32,3,0
5,Byron Black,70.0,563.0,R,175.0,30.2,Michael Chang,50.0,722.0,R,175.0,27.8,2000-301,Auckland,Hard,A,20000110,R32,3,1
6,Magnus Gustafsson,60.0,626.0,R,185.0,33.0,Mark Nielsen,246.0,135.0,R,185.0,22.2,2000-301,Auckland,Hard,A,20000110,R32,3,0
7,Thomas Johansson,43.0,785.0,R,180.0,24.7,Glenn Weiner,334.0,88.0,R,188.0,23.7,2000-301,Auckland,Hard,A,20000110,R32,3,0
8,Sjeng Schalken,37.0,843.0,R,193.0,23.3,Goran Ivanisevic,68.0,571.0,L,193.0,28.3,2000-301,Auckland,Hard,A,20000110,R32,3,0
9,Markus Hantschk,95.0,416.0,R,188.0,22.1,Tomas Behrend,115.0,344.0,R,193.0,25.0,2000-301,Auckland,Hard,A,20000110,R32,3,1


In [18]:
# Transform categorical data into numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Drop all columns that will have negligible effect
df_transformed = df_transformed.drop(columns=[
    'tourney_id',
    'tourney_name',
    'tourney_date',
    'player0_name',
    'player1_name'
])

# Convert categorical columns to integers
df_transformed['player0_hand'] = le.fit_transform(df_transformed['player0_hand'])
df_transformed['player1_hand'] = le.fit_transform(df_transformed['player1_hand'])
df_transformed['surface'] = le.fit_transform(df_transformed['surface'])
df_transformed['tourney_level'] = le.fit_transform(df_transformed['tourney_level'])
df_transformed['round'] = le.fit_transform(df_transformed['round'])
df_transformed['best_of'] = le.fit_transform(df_transformed['best_of'])

df_transformed.head(10)


,player0_rank,player0_rank_points,player0_hand,player0_ht,player0_age,player1_rank,player1_rank_points,player1_hand,player1_ht,player1_age,surface,tourney_level,round,best_of,label
0,11.0,1612.0,1,188.0,21.7,63.0,595.0,0,180.0,31.1,3,0,6,0,0
1,49.0,723.0,0,183.0,24.3,211.0,157.0,1,190.0,24.5,3,0,6,0,1
2,48.0,726.0,1,175.0,21.3,59.0,649.0,1,173.0,26.5,3,0,6,0,0
3,45.0,768.0,1,183.0,19.9,61.0,616.0,1,185.0,18.4,3,0,6,0,0
4,167.0,219.0,1,180.0,27.3,34.0,873.0,1,185.0,23.7,3,0,6,0,0
5,70.0,563.0,1,175.0,30.2,50.0,722.0,1,175.0,27.8,3,0,6,0,1
6,60.0,626.0,1,185.0,33.0,246.0,135.0,1,185.0,22.2,3,0,6,0,0
7,43.0,785.0,1,180.0,24.7,334.0,88.0,1,188.0,23.7,3,0,6,0,0
8,37.0,843.0,1,193.0,23.3,68.0,571.0,0,193.0,28.3,3,0,6,0,0
9,95.0,416.0,1,188.0,22.1,115.0,344.0,1,193.0,25.0,3,0,6,0,1


In [19]:
# Now generate new features
df_transformed['rank_diff'] = df_transformed['player0_rank'] - df_transformed['player1_rank']
df_transformed['rank_points_diff'] = df_transformed['player0_rank_points'] - df_transformed['player1_rank_points']
df_transformed['ht_diff'] = df_transformed['player0_ht'] - df_transformed['player1_ht']
df_transformed['age_diff'] = df_transformed['player0_age'] - df_transformed['player1_age']

# Drop columns that have been transformed except for rank/rank points
df_transformed = df_transformed.drop(columns=[
    'player0_ht',
    'player1_ht',
    'player0_age',
    'player1_age'
])

df_transformed.head(10)

,player0_rank,player0_rank_points,player0_hand,player1_rank,player1_rank_points,player1_hand,surface,tourney_level,round,best_of,label,rank_diff,rank_points_diff,ht_diff,age_diff
0,11.0,1612.0,1,63.0,595.0,0,3,0,6,0,0,-52.0,1017.0,8.0,-9.4
1,49.0,723.0,0,211.0,157.0,1,3,0,6,0,1,-162.0,566.0,-7.0,-0.2
2,48.0,726.0,1,59.0,649.0,1,3,0,6,0,0,-11.0,77.0,2.0,-5.2
3,45.0,768.0,1,61.0,616.0,1,3,0,6,0,0,-16.0,152.0,-2.0,1.5
4,167.0,219.0,1,34.0,873.0,1,3,0,6,0,0,133.0,-654.0,-5.0,3.6
5,70.0,563.0,1,50.0,722.0,1,3,0,6,0,1,20.0,-159.0,0.0,2.4
6,60.0,626.0,1,246.0,135.0,1,3,0,6,0,0,-186.0,491.0,0.0,10.8
7,43.0,785.0,1,334.0,88.0,1,3,0,6,0,0,-291.0,697.0,-8.0,1.0
8,37.0,843.0,1,68.0,571.0,0,3,0,6,0,0,-31.0,272.0,0.0,-5.0
9,95.0,416.0,1,115.0,344.0,1,3,0,6,0,1,-20.0,72.0,-5.0,-2.9


In [20]:
# Scale transformed data
from sklearn.preprocessing import StandardScaler
import joblib

# Select the numeric columns
numeric_columns = ['player0_rank', 'player1_rank', 'player0_rank_points', 'player1_rank_points', 'rank_diff', 'rank_points_diff', 'ht_diff', 'age_diff']

# Select other numeric columns for different scaler
numeric_diff_columns = ['player0_rank', 'player1_rank', 'player0_rank_points', 'player1_rank_points', 'rank_diff', 'rank_points_diff']

# Initialize StandardScaler
scaler = StandardScaler()
scaler_diff = StandardScaler()

# Create a copy to use a different scaler
df_copy = df_transformed.copy(deep=True)

# Scale the numeric columns
df_transformed[numeric_columns] = scaler.fit_transform(df_transformed[numeric_columns])

# Scale the other numeric columns
df_copy[numeric_diff_columns] = scaler_diff.fit_transform(df_copy[numeric_diff_columns])

# Export transformed data to csv
df_transformed.to_csv('/Users/maxharsh/Desktop/Coding Projects/MatchNet/data/transformed_matches.csv', index=False)

# Save the scaler to disk
joblib.dump(scaler_diff, "/Users/maxharsh/Desktop/Coding Projects/MatchNet/model/scaler.pkl")

# Check the changes
df_transformed.head(10)

,player0_rank,player0_rank_points,player0_hand,player1_rank,player1_rank_points,player1_hand,surface,tourney_level,round,best_of,label,rank_diff,rank_points_diff,ht_diff,age_diff
0,-0.627933,0.168535,1,-0.157868,-0.440158,0,3,0,6,0,0,-0.367334,0.473065,0.834416,-1.734055
1,-0.284610,-0.365565,0,1.149629,-0.702265,1,3,0,6,0,1,-1.149924,0.263085,-0.725161,-0.036203
2,-0.293645,-0.363762,1,-0.193206,-0.407843,1,3,0,6,0,0,-0.075641,0.035413,0.210585,-0.958948
3,-0.320750,-0.338529,1,-0.175537,-0.427591,1,3,0,6,0,0,-0.111213,0.070332,-0.205302,0.277531
4,0.781496,-0.668361,1,-0.414067,-0.273798,1,3,0,6,0,0,0.948841,-0.304932,-0.517217,0.665084
5,-0.094880,-0.461691,1,-0.272716,-0.364159,1,3,0,6,0,1,0.144907,-0.074466,0.002642,0.443625
6,-0.185228,-0.423841,1,1.458835,-0.715430,1,3,0,6,0,0,-1.320671,0.228166,0.002642,1.993838
7,-0.338819,-0.328316,1,2.236265,-0.743556,1,3,0,6,0,0,-2.067689,0.324077,-0.829133,0.185256
8,-0.393028,-0.293470,1,-0.113696,-0.454520,0,3,0,6,0,0,-0.217930,0.126202,0.002642,-0.922039
9,0.130990,-0.550006,1,0.301523,-0.590361,1,3,0,6,0,1,-0.139671,0.033085,-0.517217,-0.534485


In [21]:
# Save second dataset that excludes player hand, player height, tourney level, round, best of, and age
# Drop columns that have been transformed except for rank/rank points
df_copy = df_copy.drop(columns=[
    'player0_hand',
    'player1_hand',
    'tourney_level',
    'round', 
    'best_of',
    'ht_diff', 
    'age_diff'
])

# Export transformed data to csv
df_copy.to_csv('/Users/maxharsh/Desktop/Coding Projects/MatchNet/data/exclude_features_matches.csv', index=False)

# Check the changes
df_copy.head(10)

,player0_rank,player0_rank_points,player1_rank,player1_rank_points,surface,label,rank_diff,rank_points_diff
0,-0.627933,0.168535,-0.157868,-0.440158,3,0,-0.367334,0.473065
1,-0.284610,-0.365565,1.149629,-0.702265,3,1,-1.149924,0.263085
2,-0.293645,-0.363762,-0.193206,-0.407843,3,0,-0.075641,0.035413
3,-0.320750,-0.338529,-0.175537,-0.427591,3,0,-0.111213,0.070332
4,0.781496,-0.668361,-0.414067,-0.273798,3,0,0.948841,-0.304932
5,-0.094880,-0.461691,-0.272716,-0.364159,3,1,0.144907,-0.074466
6,-0.185228,-0.423841,1.458835,-0.715430,3,0,-1.320671,0.228166
7,-0.338819,-0.328316,2.236265,-0.743556,3,0,-2.067689,0.324077
8,-0.393028,-0.293470,-0.113696,-0.454520,3,0,-0.217930,0.126202
9,0.130990,-0.550006,0.301523,-0.590361,3,1,-0.139671,0.033085
